# pandas 进阶

前面的章节着重介绍了不同类型的数据整理工作流以及NumPy，熊猫和其他库的功能。随着时间的流逝，熊猫为高级用户开发了许多功能。本章将深入探讨一些高级功能区域，以帮助您加深作为熊猫用户的专业知识。

## 12.1 分类数据

本节介绍熊猫的分类类型。我将展示如何通过使用它在某些熊猫操作中实现更好的性能和内存使用。我还介绍了一些在统计和机器学习应用程序中使用分类数据的工具。

### 背景和动机

通常，表中的一列可能包含较小的一组不同值的重复实例。我们已经看到了诸如unique和value_counts之类的函数，这些函数使我们能够从数组中提取不同的值并分别计算其频率：

In [1]:
import numpy as np
import pandas as pd

In [2]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)

In [3]:
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [4]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [5]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

许多数据系统（用于数据仓库，统计计算或其他用途）已开发出专门的方法来表示具有重复值的数据，以实现更有效的存储和计算。在数据仓库中，最佳实践是使用包含不同值的所谓维表，并将主要观察结果存储为引用维表的整数键：

In [6]:
values = pd.Series([0, 1, 0, 0] * 2)

In [7]:
dim = pd.Series(['apple', 'orange'])

In [8]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [9]:
dim

0     apple
1    orange
dtype: object

我们可以使用 `take` 方法来恢复原始的字符串序列：

In [10]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

这种表示为整数的表示形式被称为分类或字典编码表示形式。不同值的数组可以称为数据的类别、字典或级别。在本书中，我们将使用“categorical”和“categories”一词。引用类别的整数值称为类别代码或简称为代码。

在执行分析时，分类表示形式可以显着提高性能。您还可以在不修改代码的情况下对类别执行转换。可以以相对较低的成本进行的一些示例转换是：

- 重命名类别
- 追加新类别而不改变现有类别的顺序或位置

### pandas 中的 Categorical 类型

pandas具有特殊的分类类型，用于保存使用基于整数的分类表示形式或编码的数据。让我们考虑一下之前的示例系列：

In [11]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2

In [12]:
N = len(fruits)

In [13]:
df = pd.DataFrame({'fruit': fruits, 
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])

In [14]:
df

,basket_id,fruit,count,weight
0,0,apple,4,3.853516
1,1,orange,4,3.771271
2,2,apple,9,2.283486
3,3,apple,5,2.353608
4,4,apple,10,0.846897
5,5,orange,14,1.853497
6,6,apple,7,3.413223
7,7,apple,11,3.311195


在这里，`df['fruit']` 是Python字符串对象的数组。我们可以通过调用将其转换为分类的：

In [15]:
fruit_cat = df['fruit'].astype('category')

In [16]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

`fruit_cat` 的值不是 Numpy 数组，而是 `pandas.Categorical` 实例：

In [17]:
c = fruit_cat.values

In [18]:
type(c)

pandas.core.arrays.categorical.Categorical

`Categorical` 对象拥有 `categories` 和 `codes` 属性：

In [19]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [20]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

您可以通过分配转换后的结果将 DataFrame 列转换为 `categorical`：

In [21]:
df['fruit'] = df['fruit'].astype('category')

In [22]:
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

你也可以直接从其他类型的Python序列创建 `pandas.Categorical`：

In [23]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

In [24]:
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

如果您从其他来源获得了分类编码数据，则可以使用替代 `from_codes` 构造函数：

In [31]:
categories = ['foo', 'bar', 'baz']

In [26]:
codes = [0, 1, 2, 0, 0, 1]

In [27]:
my_cats_2 = pd.Categorical.from_codes(codes, categories)

In [28]:
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

除非明确指定，否则类别转换不假定类别的特定顺序。因此，根据输入数据的顺序，类别数组可以采用不同的顺序。当使用from_codes或任何其他构造函数时，可以指示类别具有有意义的顺序：

In [34]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)

In [35]:
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

输出 `[foo <bar <baz]` 表示 "foo" 在顺序中位于 "bar" 之前，依此类推。可以使用 `as_ordered` 使无序的类别实例有序：

In [36]:
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

最后一点，即使我仅展示了字符串示例，分类数据也不必是字符串。分类数组可以由任何不可变值类型组成。

### Categorical 计算

与未编码的版本（如字符串数组）相比，在pandas中使用“Categorical”通常的行为方式相同。pandas的某些部分（例如groupby函数）在处理分类时会表现更好。还有一些功能可以利用 ordered 标志。

让我们考虑一些随机数值数据，并使用pandas.qcut分级功能。 此返回熊猫。 我们在本书的前面使用了pandas.cut，但忽略了分类工作原理的细节：

In [37]:
np.random.seed(12345)

In [38]:
draws = np.random.randn(1000)

In [39]:
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

让我们计算此数据的四分位数合并并提取一些统计信息：

In [40]:
bins = pd.qcut(draws, 4)

In [41]:
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

虽然有用，但确切的样本四分位数对生成报告的作用可能不及四分位数名称。 我们可以使用qcut的labels参数来实现此目的：

In [42]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

In [43]:
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [45]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

带有标签的分类容器不包含有关数据中容器边缘的信息，因此我们可以使用groupby提取一些摘要统计信息：

In [46]:
bins = pd.Series(bins, name='quartile')

In [48]:
results = (pd.Series(draws).groupby(bins).agg(['count', 'min', 'max']).reset_index())

In [49]:
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


结果中的“四分位数”列保留来自bin的原始分类信息，包括排序：

In [55]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

#### 使用 categoricals 获得更好性能

如果您对特定数据集进行了大量分析，则转换为分类数据可以显着提高整体性能。DataFrame列的分类版本通常也将使用更少的内存。让我们考虑一些包含1000万个元素和少量不同类别的系列：

In [56]:
N = 10000000

In [57]:
draws = pd.Series(np.random.randn(N))

In [58]:
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [59]:
categories = labels.astype('category')

现在，我们注意到标签使用的内存远远大于类别：

In [61]:
labels.memory_usage()

80000128

In [62]:
categories.memory_usage()

10000320

当然，转换为类别不是免费的，但这是一次性费用：

In [63]:
%time _ = labels.astype('category')

CPU times: user 378 ms, sys: 24.7 ms, total: 403 ms
Wall time: 425 ms


使用分类法，GroupBy操作可以显着提高速度，因为基础算法使用基于整数的代码数组而不是字符串数组。

### Categorical 方法

包含分类数据的系列具有几种类似于Series.str专用字符串方法的特殊方法。这也提供了对类别和代码的便捷访问。考虑系列：

In [64]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)

In [65]:
cat_s = s.astype('category')

In [66]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

特殊属性cat允许您访问分类方法：

In [70]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [72]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

假设我们知道该数据的实际类别集合超出了数据中观察到的四个值。我们可以使用set_categories方法来更改它们：

In [73]:
actual_categories = ['a', 'b', 'c', 'd', 'e']

In [75]:
cat_s2 = cat_s.cat.set_categories(actual_categories)

In [76]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

尽管数据看起来没有变化，但是新类别将反映在使用它们的操作中。例如，value_counts尊重类别（如果存在）：

In [77]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [78]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

在大型数据集中，分类通常用作节省内存和提高性能的便捷工具。筛选大的DataFrame或Series之后，许多类别可能不会出现在数据中。为了解决这个问题，我们可以使用remove_unused_categories方法修剪未观察到的类别：

In [79]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]

In [80]:
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [83]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

有关可用分类方法的列表，请参见表12-1。

#### 创建虚拟变量以进行建模

在使用统计信息或机器学习工具时，通常会将分类数据转换为虚拟变量，也称为“一键编码”。这涉及到为每个不同的类别创建一个带有一列的DataFrame； 对于给定的类别，这些列包含1，否则包含0。

考虑前面的示例：

In [85]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

如先前在第7章中所述，pandas.get_dummies函数将此一维分类数据转换为包含虚拟变量的DataFrame：

In [86]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## 12.2 GroupBy 进阶使用

尽管我们已经在第10章中对Series和DataFrame的groupby方法进行了深入讨论，但您可能会发现还有一些其他使用技巧。

### 组转换和“展开的” GroupBy

在第10章中，我们研究了分组操作中的apply方法来执行转换。还有另一种称为transform的内置方法，该方法与apply类似，但是对您可以使用的函数类型施加了更多约束：

- 它可以产生标量值以广播到组的形状
- 它可以产生与输入组相同形状的对象
- 它不能改变其输入

让我们考虑一个简单的示例：

In [88]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})

In [89]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


以下是按键的分组均值：

In [94]:
g = df.groupby('key').value

In [95]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

假设我们想产生一个与df['value']形状相同的Series，但其值由'key'分组的平均值代替。我们可以传递函数lambda x：x.mean（）进行转换：

In [99]:
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

对于内置的聚合函数，我们可以像GroupBy agg方法一样传递字符串别名：

In [100]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

像apply一样，transform也可以使用返回Series的函数，但是结果的大小必须与输入的大小相同。例如，我们可以使用lambda函数将每个组乘以2：

In [104]:
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

作为更复杂的示例，我们可以按降序计算每个组的等级：

In [106]:
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

考虑由简单聚合组成的组转换函数：

In [107]:
def normalize(x):
    return (x - x.mean()) / x.std()

在这种情况下，我们可以使用transform或apply获得等效的结果：

In [108]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [109]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

内置的聚合函数（例如“均值”或“求和”）通常比常规应用函数要快得多。与转换一起使用时，它们也具有“快速过去”的功能。这使我们可以执行所谓的未包装的组操作：

In [110]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [114]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')

In [115]:
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

尽管展开的组操作可能涉及多个组聚合，但是矢量化操作的总体优势通常胜过此。

### 分组时间重采样

对于时间序列数据，重采样方法在语义上是基于时间间隔的分组操作。这是一个小的示例表：

In [116]:
N = 15

In [118]:
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)

In [119]:
df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})

In [120]:
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


在这里，我们可以按“时间”编制索引，然后重新采样：

In [122]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


假设一个DataFrame包含多个时间序列，并用附加的组键列标记：

In [125]:
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})

In [126]:
df2[:7]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


为了对每个“键”值进行相同的重采样，我们引入了pandas.TimeGrouper对象：

In [133]:
time_key = pd.Grouper(freq='5min')

然后，我们可以设置时间索引，按“键”和“ time_key”分组，然后进行汇总：

In [134]:
resampled = (df2.set_index('time').groupby(['key', time_key]).sum())

In [135]:
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [136]:
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0


使用TimeGrouper的一个限制是时间必须是Series或DataFrame的索引。